In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [2]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.2/691.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [3]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
import keras
import keras_nlp

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

In [7]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

In [27]:
print(len(fewshot_examples))

2458


In [32]:
#@title Give me a Prompt

from google.colab import files
import os
import re
import hashlib
import random

from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

Prompt = 'Give me a single dense layer of 64 units, ReLU activation, and then an output layer.' #@param {type:"string"}

fewshot_examples = """
ROLE: You are an AI assistant helping researchers define custom neural network architectures for protein sequence analysis. Your task is to generate One YAML file based on the user's specifications for their model according to the exmaples below.

The YAML file will contain a list of layers that compose a neural network. Each layer will include the function (e.g., fully connected, dropout) and its associated parameters. The user will describe the kind of model they want to build in natural language, and you will need to convert that into the correct YAML format.

EXAMPLES:

Example 1:
**User request:**
"I want a neural network with a fully connected layer with 512 units, followed by a dropout layer with a rate of 0.2, and a final output layer with 10 units."

**Response: **
```yaml
network:
  - layer_func: ~tf.layers.dense
    arguments:
      name: dense1
      inputs: ~layers[-1]
      units: 512
      activation: ~tf.nn.relu

  - layer_func: ~tf.layers.dropout
    arguments:
      inputs: ~layers[-1]
      rate: 0.2
      training: ~ph_inputs_dict["training"]

  - layer_func: ~tf.layers.dense
    arguments:
      name: output_layer
      inputs: ~layers[-1]
      units: 10
      activation: ~tf.nn.softmax

Example 2:
**User request:**
"I want a simple convolutional network with 64 filters, a kernel size of 3, followed by a max-pooling layer, and a dense layer with 128 units."

**Response**
```yaml
network:
  - layer_func: ~tf.layers.conv2d
    arguments:
      name: conv1
      inputs: ~layers[-1]
      filters: 64
      kernel_size: [3, 3]
      activation: ~tf.nn.relu

  - layer_func: ~tf.layers.max_pooling2d
    arguments:
      inputs: ~layers[-1]
      pool_size: [2, 2]
      strides: [2, 2]

  - layer_func: ~tf.layers.dense
    arguments:
      name: dense1
      inputs: ~layers[-1]
      units: 128
      activation: ~tf.nn.relu

Example 3:
**User request:**
"I need a network with a graph convolution layer followed by a dense layer with 256 units, using the LeakyReLU activation."

**Response**
```yaml
network:
  - layer_func: ~bg_inference.graph_convolution
    arguments:
      inputs: ~layers[-1]
      adj_mtx: ~ph_inputs_dict["adj_mtx"]
      units: 256
      activation: ~tf.nn.leaky_relu

  - layer_func: ~tf.layers.dense
    arguments:
      name: dense1
      inputs: ~layers[-1]
      units: 256
      activation: ~tf.nn.leaky_relu

ACTION: Just give me a reponse without giving anyother examples
"""

# prompt_input = input()

prompt = template.format(
    instruction= fewshot_examples + Prompt,
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
output = gemma_lm.generate(prompt)
Prompt_len = len(Prompt)
print(output[2470 + Prompt_len + 15:])

# Give me a single dense layer of 64 units, ReLU activation, and then an output layer.

network:
  - layer_func: ~tf.layers.dense
    arguments:
      name: dense1
      inputs:
        tf_inputs: ~layers[-1]
      units: 64
      activation: ~tf.nn.relu
  
  - layer_func: ~tf.layers.dense
    arguments:
      name: output_layer
      inputs: ~layers[-2]
      units: 10
      activation: ~tf.nn.softmax

```

